In [3]:
import sys
sys.path.append('../src')
import os
import numpy as np
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib notebook
import FASTSim_BX_v2_ECMS_rulebase


##################################################################################
# Individual Drive Cycle
# Load Drive Cycle
# Default (UDDS, US06, HWFET) cycles can be loaded from the ../cycles directory, or custom cycles can be specified in the same format. The expected format is a dictionary with the following keys:
# ['cycGrade', 'cycMps', 'cycSecs', 'cycRoadType']
#     cycGrade = Road grade [%/100]
#     cycMps = Vehicle speed [meters per second]
#     cycSecs = Relative time in the cycles [seconds]
#     cycRoadType = Indicator as to whether or not there is a wireless charging capability from the road to vehicle
# There is no limit to the length of a drive cycle that can be provided as an input to FASTSim.
cyc = FASTSim_BX_v2_ECMS_rulebase.get_standard_cycle("UDDS")


##################################################################################
# choose Powertrain Model ########################################################
##################################################################################
# A vehicle database in CSV format is required to be in the working directory where FASTSim is running 
# (i.e. the same directory as this notebook). The "get_veh" function selects the appropriate vehicle
# attributes from the database and contructs the powertrain model (engine efficiency map, etc.). 
# An integer value corresponds to each vehicle in the database. To add a new vehicle, simply 
# populate a new row to the vehicle database CSV.

### Toyota Mirai fuel cell
veh_num=24

### Hyndai Tucson fuel cell
# veh_num=23


veh = FASTSim_BX_v2_ECMS_rulebase.get_veh(veh_num)
print(veh['vehPtType'],'/',veh['name'])


########################################################################################
# choose baseline EMS strategy #########################################################
########################################################################################
#veh['EMS']='ECMS'
veh['EMS']='rule'


########################################################################################
# run the simulation
# Run FASTSim
# The "sim_drive" function takes the drive cycle and vehicle models defined above as inputs. The output is a dictionary of time series and scalar values described the simulation results. Typically of interest is the "gge" key, which is an array of time series energy consumption data at each time step in the drive cycle. Additionally, to add a result from the simulator to the output dictionary, a user can modify the sim_drive_sub function source code to include the desired field.
# If running FASTSim in batch over many drive cycles, the output from "sim_drive" can be written to files or database for batch post-processing.
output = FASTSim_BX_v2_ECMS_rulebase.sim_drive(cyc, veh)
   
    
########################################################################################
# Results
df = pd.DataFrame.from_dict(output)[['soc','fcKwInAch','fcKwOutAch','essKwOutAch',\
                                     'mcMechKwOutAch','auxInKw','mcElecKwInAch','mpgge',\
                                     'transKwInAch','mcMechKw4ForcedFc','canPowerAllElectrically',\
                                     'fuelKg','fuelKgAch']]
df['speed'] = cyc['cycMps'] * 2.23694  # Convert mps to mph
print('MPG is: ',df['mpgge'][-1:])


########################################################################################
#  plot
ax = plt.subplot(3,1,1)
speed_line = df.speed.plot(color='xkcd:pale red', ax=ax, label='Speed')
ax.set_ylabel('Speed [MPH]', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')

ax = plt.subplot(3,1,2)
line1 = df.fcKwOutAch.plot(ax=ax, label='kW')
line2 = df.essKwOutAch.plot(ax=ax, label='kW')
ax.set_ylabel('Power [kW]', weight='bold', color='xkcd:bluish')
ax.tick_params('y', colors='xkcd:bluish')
ax.legend(['fuel cell','Li-ion battery'])

ax11 = plt.subplot(3,1,3)
speed_line = df.fuelKgAch.plot(color='xkcd:pale red', ax=ax11, label='Speed')
ax11.set_xlabel('Cycle Time [s]', weight='bold')
ax11.set_ylabel('fuel burned [kg]', weight='bold', color='xkcd:bluish')
ax11.tick_params('y', colors='xkcd:bluish')

2.0 / Toyota Mirai
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMa

curMaxFcKwOut: 23.966467069869626 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 56.666467069869626 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.966467069869626 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 56.666467069869626 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw'

curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.245308291193044 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.945308291193044 veh['mcMaxElecInKw']: 121.50537634408602
curM

curMaxElecKw: 58.97576816042529 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.275768160425287 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.97576816042529 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.275768160425287 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.97576816042529 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.275768160425287 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.97576816042529 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.275768160425287 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.97576816042529 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.505376

curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730742 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 62.1750333730742 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 29.4750333730

...

curMaxFcKwOut: 26.20661457844404 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.90661457844404 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFc

curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 22.8 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.5 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 

curMaxFcKwOut: 26.893032135415304 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 59.59303213541531 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.67263552158591 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 59.37263552158591 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 27.072656157148252 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 59.772656157148255 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 26.257951235841595 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.9579512358416 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 25.972923525045687 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.67292352504569 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 25.340270878582142 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 58.04027087858215 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwO

curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElecKw: 55.800000000000004 veh['mcMaxElecInKw']: 121.50537634408602
curMaxFcKwOut: 23.1 curMaxRoadwayChgKw: 0.0 curMaxEssKwOut: 33.0 auxInKw: 0.3
curMaxElec

<IPython.core.display.Javascript object>

In [8]:
veh

{'selection': 24.0,
 'name': 'Toyota Mirai',
 'vehPtType': 2.0,
 'dragCoef': 0.3,
 'frontalAreaM2': 2.786,
 'gliderKg': 1171.581,
 'vehCgM': 0.53,
 'driveAxleWeightFrac': 0.59,
 'wheelBaseM': 2.78,
 'cargoKg': 136.0,
 'vehOverrideKg': '',
 'maxFuelStorKw': 1000.0,
 'fuelStorSecsToPeakPwr': 1.0,
 'fuelStorKwh': 168.5,
 'fuelStorKwhPerKg': 1.91,
 'maxFuelConvKw': 114.0,
 'fcEffType': 4.0,
 'fcAbsEffImpr': 0.0,
 'fuelConvSecsToPeakPwr': 5.0,
 'fuelConvBaseKg': 0.0,
 'fuelConvKwPerKg': 2.04,
 'maxMotorKw': 113.0,
 'motorPeakEff': 0.95,
 'motorSecsToPeakPwr': 3.0,
 'mcPeKgPerKw': 0.833,
 'mcPeBaseKg': 21.6,
 'maxEssKw': 33.0,
 'maxEssKwh': 1.6,
 'essKgPerKwh': 44.0,
 'essBaseKg': 0.0,
 'essRoundTripEff': 0.97,
 'essLifeCoefA': 110.0,
 'essLifeCoefB': -0.6844,
 'wheelInertiaKgM2': 0.815,
 'numWheels': 4.0,
 'wheelRrCoef': 0.0076,
 'wheelRadiusM': 0.334,
 'wheelCoefOfFric': 0.7,
 'minSoc': 0.4,
 'maxSoc': 0.8,
 'essDischgToFcMaxEffPerc': 0.0,
 'essChgToFcMaxEffPerc': 0.0,
 'maxAccelBufferMph'